In [10]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [11]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [12]:
downloaded = drive.CreateFile({'id':'1fgdAW1wEXkPn7ojAec5web9PnJL3f3kL'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Library.xml')

In [13]:
# You can use this code from the link below as a starting point for your application: http://www.py4e.com/code3/tracks.zip

import xml.etree.ElementTree as ET
import sqlite3
 
conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()
 
# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Genre;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
 
CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);
 
CREATE TABLE Genre(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT UNIQUE
);
 
CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);
 
CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')
 
fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Library.xml'
 
# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
# <key>Genre</key><string>Industrial</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None
 
#create element tress and find branch
stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')#add one dict/
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue
 
    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    genre = lookup(entry, 'Genre')#add
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')
 
    if name is None or artist is None or album is None or genre is None: 
        continue
 
    print(name, artist, genre, album, count, rating, length)
 
    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]
 
    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]
 
    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]
 
    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, album_id, genre_id, length, rating, count ) )
 
    conn.commit()

Enter file name: Library.xml
Dict count: 404
Another One Bites The Dust Queen Rock Greatest Hits 55 100 217103
Asche Zu Asche Rammstein Industrial Herzeleid 79 100 231810
Beauty School Dropout Various Soundtrack Grease 48 100 239960
Black Dog Led Zeppelin Rock IV 109 100 296620
Bring The Boys Back Home Pink Floyd Rock The Wall [Disc 2] 33 100 87118
Circles Bryan Lee Funk Blues Is 54 60 355369
Comfortably Numb Pink Floyd Rock The Wall [Disc 2] 36 100 384130
Crazy Little Thing Called Love Queen Rock Greatest Hits 38 100 163631
Electric Funeral Black Sabbath Metal Paranoid 44 100 293015
Fat Bottomed Girls Queen Rock Greatest Hits 38 100 257515
For Those About To Rock (We Salute You) AC/DC Rock Who Made Who 84 100 353750
Four Sticks Led Zeppelin Rock IV 84 100 284421
Furious Angels Rob Dougan Soundtrack The Matrix Reloaded 54 100 330004
Gelle Bryan Lee Blues/R&B Blues Is 45 60 199836
Going To California Led Zeppelin Rock IV 100 100 215666
Grease Various Soundtrack Grease 42 100 205792
Hand